In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# IMDb En İyiler sayfasından URL'leri çek
main_url = "https://www.hdfilmcehennemi.men/imdb-en-iyiler/"
response = requests.get(main_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Film URL'lerini çek
base_url = "https://www.hdfilmcehennemi.men"
film_links = soup.select("div.top-imdb-list.tleft a")
urls = [link["href"] for link in film_links if link["href"].startswith("http") or link["href"].startswith("/film/")]
urls = [url if url.startswith("http") else base_url + url for url in urls]

print(f"{len(urls)} adet film bulundu.")

# Film bilgilerini çek
data = []

for url in urls:
    print(f"Scraping: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    def get_text(selector, multiple=False, separator=', '):
        try:
            if multiple:
                return separator.join([el.text.strip() for el in soup.select(selector)])
            element = soup.select_one(selector)
            return element.text.strip() if element else None
        except:
            return None

    def get_next_to_label(label):
        try:
            return soup.find('b', string=label).find_next_sibling('span').text.strip()
        except:
            return None

    title     = get_text('div.extra .or-title')
    year      = get_text('div.extra span.C a')
    runtime   = get_text('span.runtime')
    imdb      = soup.find('b', string="IMDb Puanı")
    imdb_rate = imdb.find_next('strong').text.strip() if imdb else None
    director  = get_next_to_label("Yönetmen")
    genres    = get_text('div.sgeneros a', multiple=True)
    genre     = genres.split(",")[0] if genres else None  # sadece ilk türü al

    data.append({
        'Title': title,
        'Genre': genre,
        'Runtime': runtime,
        'Year': year,
        'Rating': imdb_rate,
        'Director': director
    })

# Excel çıktısı
df = pd.DataFrame(data)
df.to_excel('imdb_top_ozet.xlsx', index=False)

print("✅ 'imdb_top_ozet.xlsx' başarıyla oluşturuldu.")


250 adet film bulundu.
Scraping: https://www.hdfilmcehennemi.men/film/esaretin-bedeli-1994-izle/
Scraping: https://www.hdfilmcehennemi.men/film/baba-1972-izle/
Scraping: https://www.hdfilmcehennemi.men/film/kara-sovalye-2008-izle/
Scraping: https://www.hdfilmcehennemi.men/film/yuzuklerin-efendisi-3-2003-izle/
Scraping: https://www.hdfilmcehennemi.men/film/schindlerin-listesi-1993-izle/
Scraping: https://www.hdfilmcehennemi.men/film/baba-2-1974-izle/
Scraping: https://www.hdfilmcehennemi.men/film/12-kizgin-adam-1957-izle/
Scraping: https://www.hdfilmcehennemi.men/film/ucuz-roman-1994-izle/
Scraping: https://www.hdfilmcehennemi.men/film/yuzuklerin-efendisi-1-2001-izle/
Scraping: https://www.hdfilmcehennemi.men/film/baslangic-2010-izle/
Scraping: https://www.hdfilmcehennemi.men/film/dovus-kulubu-1999-izle/
Scraping: https://www.hdfilmcehennemi.men/film/forrest-gump-1994-izle/
Scraping: https://www.hdfilmcehennemi.men/film/yuzuklerin-efendisi-2-2002-izle/
Scraping: https://www.hdfilmcehenn